## MODEL DOWNLOAD

In [1]:
from transformers import DistilBertTokenizer, DistilBertModel

model_name = "distilbert-base-uncased"

# Downloads & caches locally (usually in ~/.cache/huggingface/)
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\typis\PycharmProjects\Classwork\venv310\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\typis\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
tokenizer.save_pretrained("./distilbert_local")
model.save_pretrained("./distilbert_local")

## DATASET LOADING

In [30]:
import pandas as pd
import numpy as np
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer

In [45]:
movies = pd.read_csv('movies_sub.csv')
u_item = pd.read_csv("u_item.csv", encoding="ISO-8859-1")

In [3]:
u_item.columns

Index(['movie id', 'movie title', 'release date', 'video release date',
       'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children's',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film Noir',
       'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western'],
      dtype='object')

In [4]:
u_item.rename(columns={'movie id': 'Movie_ID'}, inplace=True)

In [5]:
genre_cols = ['Action', 'Adventure', 'Animation', "Children's", 'Comedy',
              'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film Noir',
              'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
              'Thriller', 'War', 'Western']

data = movies[['Movie_ID', 'Movie_Name', 'Llama Summary Few Shot']].merge(u_item[['Movie_ID'] + genre_cols], on='Movie_ID')
data.head()

,Movie_ID,Movie_Name,Llama Summary Few Shot,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),"In a world where toys come to life, a showdown...",0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),"When a secret weapon system, GoldenEye, is det...",1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),"At the mysterious Mon Senor hotel, a young bel...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),"In the town of Hollywood, where fame and wealt...",1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),"In a chilling game of cat and mouse, a cunning...",0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
#data = data.sample(n=100, random_state=42).reset_index(drop=True)

texts = data['Llama Summary Few Shot'].tolist()
labels = data[genre_cols].values

## MODEL IMPLEMENTATION

In [22]:
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = DistilBertTokenizer.from_pretrained("./distilbert_local")
model = DistilBertForSequenceClassification.from_pretrained("./distilbert_local", num_labels=len(genre_cols)).to(device)

inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)

# Move inputs to GPU if available
inputs = {key: value.to(device) for key, value in inputs.items()}
labels = torch.tensor(labels).to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at ./distilbert_local and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
inputs

{'input_ids': tensor([[ 101, 1999, 1037,  ...,    0,    0,    0],
         [ 101, 2043, 1037,  ...,    0,    0,    0],
         [ 101, 2012, 1996,  ...,    0,    0,    0],
         ...,
         [ 101, 1999, 1996,  ...,    0,    0,    0],
         [ 101, 1999, 1037,  ...,    0,    0,    0],
         [ 101, 1037, 2136,  ...,    0,    0,    0]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}

### TRAINING

In [77]:
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR

# Prepare data loader
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels)
train_dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=4e-5)
class_counts = labels.sum(dim=0)
pos_weights = (len(labels) - class_counts) / (class_counts + 1e-5)
loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weights) # Handle multi-label classification correctly.

# Initialize scheduler
scheduler = StepLR(optimizer, step_size=2, gamma=0.1)  # Adjust the learning rate every 2 epochs

# Early stopping setup
best_loss = float('inf')
patience_counter = 0
patience = 3  # Stop training if no improvement in loss for 2 consecutive epochs

# Start training
model.train()
epochs = 15
for epoch in range(epochs):
    total_loss = 0
    for batch in train_dataloader:
        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        input_ids, attention_mask, label = batch
        output = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(output.logits, label.float())
        
        # Backward pass
        loss.backward()

        # Optimizer step
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

    # Step the scheduler to adjust learning rate
    scheduler.step()

    # Early stopping check
    if avg_loss < best_loss:
        best_loss = avg_loss
        patience_counter = 0  # Reset counter
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered!")
            break

Epoch 1/15 - Loss: 1.1085
Epoch 2/15 - Loss: 0.8057
Epoch 3/15 - Loss: 0.6569
Epoch 4/15 - Loss: 0.6135
Epoch 5/15 - Loss: 0.5983
Epoch 6/15 - Loss: 0.5899
Epoch 7/15 - Loss: 0.5905
Epoch 8/15 - Loss: 0.5975
Epoch 9/15 - Loss: 0.5884
Epoch 10/15 - Loss: 0.5921
Epoch 11/15 - Loss: 0.5894
Epoch 12/15 - Loss: 0.5893
Early stopping triggered!


In [78]:
best_loss

0.5883693272627674

In [36]:
model.save_pretrained("./distilbert_finetuned_local")
tokenizer.save_pretrained("./distilbert_finetuned_local")

('./distilbert_finetuned_local\\tokenizer_config.json',
 './distilbert_finetuned_local\\special_tokens_map.json',
 './distilbert_finetuned_local\\vocab.txt',
 './distilbert_finetuned_local\\added_tokens.json')

### PREDICTION

In [37]:
from tqdm import tqdm

predictions = []

with torch.no_grad(): # Disables gradient tracking since we’re just predicting.
    for text in tqdm(texts, desc="Predicting genres"):
        encoded = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512).to("cuda")
        outputs = model(**encoded)
        probs = torch.sigmoid(outputs.logits)
        preds = (probs > 0.5).int().squeeze().tolist()
        predictions.append(preds)

df = pd.DataFrame(predictions, columns=genre_cols)
df = pd.concat([data[["Movie_ID", "Movie_Name"]], df], axis=1)

df.head()

Predicting genres: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1633/1633 [00:12<00:00, 127.21it/s]


,Movie_ID,Movie_Name,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,1,1,1,0,1,0,0,1,0,1,1,0,1,0,1,1,1
1,2,GoldenEye (1995),0,1,1,0,0,1,0,0,1,1,1,1,0,1,0,0,1,1
2,3,Four Rooms (1995),0,1,1,1,0,1,0,0,1,0,1,1,0,1,0,0,1,1
3,4,Get Shorty (1995),0,0,1,1,0,1,0,0,1,1,1,1,0,1,0,0,1,1
4,5,Copycat (1995),0,1,1,1,0,1,0,0,1,1,1,1,0,1,0,0,1,1


### CLASSIFICATION REPORT

In [38]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Ensure movies and df have the same genre columns
genres_list = [
    "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", 
    "Documentary", "Drama", "Fantasy", "Film Noir", "Horror", "Musical", 
    "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]

# Initialize result dictionary
results = {"Genre": [], "Precision": [], "Recall": [], "F1-score": []}

# Compute metrics for each genre
for genre in genres_list:
    precision = precision_score(data[genre], df[genre])
    recall = recall_score(data[genre], df[genre])
    f1 = f1_score(data[genre], df[genre])

    # Store results rounded to 2 decimal places
    results["Genre"].append(genre)
    results["Precision"].append(round(precision, 2))
    results["Recall"].append(round(recall, 2))
    results["F1-score"].append(round(f1, 2))

# Convert results into a DataFrame
metrics_df = pd.DataFrame(results)

metrics_df.to_csv("DB_Classification_Score.csv", index=False)

# Display the result
print(metrics_df)


          Genre  Precision  Recall  F1-score
0        Action       0.13    0.13      0.13
1     Adventure       0.07    0.82      0.13
2     Animation       0.02    0.93      0.05
3    Children's       0.08    0.93      0.14
4        Comedy       0.22    0.04      0.07
5         Crime       0.07    1.00      0.12
6   Documentary       0.11    0.02      0.03
7         Drama       0.00    0.00      0.00
8       Fantasy       0.01    1.00      0.03
9     Film Noir       0.03    0.67      0.05
10       Horror       0.05    1.00      0.10
11      Musical       0.03    0.96      0.07
12      Mystery       0.00    0.00      0.00
13      Romance       0.16    0.96      0.27
14       Sci-Fi       0.00    0.00      0.00
15     Thriller       0.23    0.05      0.08
16          War       0.04    1.00      0.08
17      Western       0.02    0.74      0.05


C:\Users\typis\PycharmProjects\Classwork\venv310\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\typis\PycharmProjects\Classwork\venv310\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\typis\PycharmProjects\Classwork\venv310\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

## HYPER-PARAMETER TUNING

In [101]:
!pip install prettytable

In [8]:
!pip install datasets

  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp312-cp312-win_amd64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached frozenlist-1.5.0-cp312-cp312-win_amd64.whl.metadata (14 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached multiprocess-0.70.16-py312-none-any.whl (146 kB)
   ---------------------------------------- 0.0/25.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/25.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/25.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/25.3 MB ? eta -:--:--
    --------------------------------------- 0.5/25.3 MB 419.4 kB/s eta 0:00:59
    ------------------------------------

In [11]:
!pip install optuna

In [13]:
!pip install --upgrade transformers

   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.4 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.4 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.4 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.4 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.4 MB 279.8 kB/s eta 0:00:36
   -- ------------------------------------- 0.5/10.4 MB 279.8 kB/s eta 0:00:36
   -- ------------------------------------- 0.5/10.4 MB 279.8 kB/s eta 0:00:36
   --- ------------------------------------ 0.8/10.4 MB 332.2 kB/s eta 0:00:29
   --- ------------------------------------ 0.8/10.4 MB 332.2 kB/s eta 0:00:29
   --- ------------------------------------ 0.8/10.4 MB 332.2 kB/s eta 0:00:29
   --- ------------------------------------ 0.8/10.4 MB 332.2 kB/s eta 0:00:29
   --- ------------------------------------ 0.8/10.4 MB 332.2 kB/s eta 0:00:29
   ---- ----------

In [26]:
!pip install accelerate>=0.26.0

In [29]:
!pip install transformers[torch]

In [30]:
pip show accelerate

Name: accelerate
Version: 1.6.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: C:\Users\typis\PycharmProjects\Classwork\venv310\Lib\site-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [7]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = DistilBertTokenizer.from_pretrained("./distilbert_finetuned_local")
model = DistilBertForSequenceClassification.from_pretrained("./distilbert_finetuned_local", num_labels=len(genre_cols), problem_type="multi_label_classification").to(device)

# Split the data
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

train_labels = [list(map(float, lbl)) for lbl in train_labels]
val_labels = [list(map(float, lbl)) for lbl in val_labels]

# Tokenize
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Create HuggingFace datasets
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})
val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': val_labels
})

# Final dictionary
encoded_dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})


In [8]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np
import torch

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.sigmoid(torch.tensor(logits)).numpy()  # convert logits to probs
    preds = (probs > 0.5).astype(int)                     # apply threshold
    labels = np.array(labels)                            # ensure correct format

    return {
        'precision': precision_score(labels, preds, average='micro', zero_division=0),
        'recall': recall_score(labels, preds, average='micro', zero_division=0),
        'f1': f1_score(labels, preds, average='micro', zero_division=0),
        'accuracy': accuracy_score(labels, preds)
    }

In [45]:
import optuna
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
from sklearn.metrics import precision_score
import numpy as np

# Objective function for Optuna
def objective(trial):
    # Suggesting hyperparameters
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 5e-5, log=True)  # picks from range
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32])   # picks from list 
    weight_decay = trial.suggest_float('weight_decay', 0.0, 0.3)

    # Define TrainingArguments
    training_args = TrainingArguments(
        output_dir="./optuna_results",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=3,
        weight_decay=weight_decay,
        save_strategy="steps",          # Save model every few steps
        save_steps=100,                 # Save every 100 steps
        eval_strategy="steps",         # Evaluate every few steps
        eval_steps=100,                 # Evaluate every 100 steps
        logging_steps=100,              # Log every 100 steps
        load_best_model_at_end=True,    # Load best model at the end based on evaluation
        disable_tqdm=True,              # Disable tqdm to avoid overloading output
        report_to="none",               # Disable logging to platforms like TensorBoard
)

    # Trainer setup
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=encoded_dataset['train'],
        eval_dataset=encoded_dataset['validation'],
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
    )

    trainer.train()
    eval_results = trainer.evaluate()
    return eval_results["eval_precision"]  # Maximize precision

# Run Optuna
optuna.logging.set_verbosity(optuna.logging.INFO)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

print("Best Parameters:", study.best_params)


[I 2025-04-16 11:28:07,877] A new study created in memory with name: no-name-1278f138-78fe-45a7-8f32-d203e3bb91e1


{'loss': 0.1972, 'grad_norm': 0.30486366152763367, 'learning_rate': 8.887883147393474e-06, 'epoch': 2.4390243902439024}
{'train_runtime': 223.3458, 'train_samples_per_second': 17.542, 'train_steps_per_second': 0.551, 'train_loss': 0.18961629441113975, 'epoch': 3.0}


[I 2025-04-16 11:31:57,500] Trial 0 finished with value: 0.7352112676056338 and parameters: {'learning_rate': 4.5550401130391553e-05, 'batch_size': 32, 'weight_decay': 0.013398179910986284}. Best is trial 0 with value: 0.7352112676056338.


{'eval_loss': 0.16484810411930084, 'eval_precision': 0.7352112676056338, 'eval_runtime': 5.31, 'eval_samples_per_second': 61.582, 'eval_steps_per_second': 2.072, 'epoch': 3.0}
{'loss': 0.146, 'grad_norm': 0.41365566849708557, 'learning_rate': 1.8768384686755683e-05, 'epoch': 1.2195121951219512}
{'loss': 0.1222, 'grad_norm': 0.47034579515457153, 'learning_rate': 6.000776056309641e-06, 'epoch': 2.4390243902439024}
{'train_runtime': 118.792, 'train_samples_per_second': 32.982, 'train_steps_per_second': 2.071, 'train_loss': 0.12961342470432685, 'epoch': 3.0}


[I 2025-04-16 11:34:00,607] Trial 1 finished with value: 0.738498789346247 and parameters: {'learning_rate': 3.140831723089727e-05, 'batch_size': 16, 'weight_decay': 0.2206635125996341}. Best is trial 1 with value: 0.738498789346247.


...

{'eval_loss': 0.19087469577789307, 'eval_precision': 0.7021276595744681, 'eval_runtime': 2.9707, 'eval_samples_per_second': 110.077, 'eval_steps_per_second': 7.069, 'epoch': 3.0}
{'loss': 0.0242, 'grad_norm': 0.1868039220571518, 'learning_rate': 6.9114405226793635e-06, 'epoch': 2.4390243902439024}
{'train_runtime': 188.3249, 'train_samples_per_second': 20.804, 'train_steps_per_second': 0.653, 'train_loss': 0.023334283169692126, 'epoch': 3.0}


[I 2025-04-16 11:50:53,451] Trial 8 finished with value: 0.6918367346938775 and parameters: {'learning_rate': 3.5421132678731735e-05, 'batch_size': 32, 'weight_decay': 0.07951492116176469}. Best is trial 1 with value: 0.738498789346247.


{'eval_loss': 0.20247139036655426, 'eval_precision': 0.6918367346938775, 'eval_runtime': 2.8345, 'eval_samples_per_second': 115.366, 'eval_steps_per_second': 3.881, 'epoch': 3.0}
{'loss': 0.0183, 'grad_norm': 0.35307103395462036, 'learning_rate': 1.8353387782633338e-05, 'epoch': 0.6097560975609756}
{'loss': 0.0176, 'grad_norm': 0.14890389144420624, 'learning_rate': 1.3683314555500173e-05, 'epoch': 1.2195121951219512}
{'loss': 0.0182, 'grad_norm': 0.3496118187904358, 'learning_rate': 9.013241328367007e-06, 'epoch': 1.8292682926829267}
{'loss': 0.0167, 'grad_norm': 0.08236336708068848, 'learning_rate': 4.343168101233843e-06, 'epoch': 2.4390243902439024}
{'train_runtime': 122.6753, 'train_samples_per_second': 31.938, 'train_steps_per_second': 4.011, 'train_loss': 0.01706998425770581, 'epoch': 3.0}


[I 2025-04-16 11:52:59,912] Trial 9 finished with value: 0.6821862348178138 and parameters: {'learning_rate': 2.297676027749517e-05, 'batch_size': 8, 'weight_decay': 0.15087534004209116}. Best is trial 1 with value: 0.738498789346247.


{'eval_loss': 0.21954531967639923, 'eval_precision': 0.6821862348178138, 'eval_runtime': 2.7838, 'eval_samples_per_second': 117.465, 'eval_steps_per_second': 14.728, 'epoch': 3.0}
Best Parameters: {'learning_rate': 3.140831723089727e-05, 'batch_size': 16, 'weight_decay': 0.2206635125996341}


In [9]:
import json

best_params = {
    "learning_rate": 3.140831723089727e-05,
    "batch_size": 16,
    "weight_decay": 0.2206635125996341
}

#with open("best_hyperparams.json", "w") as f:
#    json.dump(best_params, f)

In [40]:
#with open("best_hyperparams.json", "r") as f:
#    best_params = json.load(f)

In [23]:
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR

inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)

# Move inputs to GPU if available
inputs = {key: value.to(device) for key, value in inputs.items()}
labels = torch.tensor(labels).to(device)

# Prepare data loader
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels)
train_dataloader = DataLoader(dataset, batch_size = best_params['batch_size'], shuffle=True)

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr = best_params['learning_rate'], weight_decay = best_params['weight_decay'])
class_counts = labels.sum(dim=0)
pos_weights = (len(labels) - class_counts) / (class_counts + 1e-5)
loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weights) # Handle multi-label classification correctly.

# Initialize scheduler
scheduler = StepLR(optimizer, step_size=2, gamma=0.1)  # Adjust the learning rate every 2 epochs

# Early stopping setup
best_loss = float('inf')
patience_counter = 0
patience = 3  # Stop training if no improvement in loss for 2 consecutive epochs

# Start training
model.train()
epochs = 15
for epoch in range(epochs):
    total_loss = 0
    for batch in train_dataloader:
        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        input_ids, attention_mask, label = batch
        output = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(output.logits, label.float())
        
        # Backward pass
        loss.backward()

        # Optimizer step
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

    # Step the scheduler to adjust learning rate
    scheduler.step()

    # Early stopping check
    if avg_loss < best_loss:
        best_loss = avg_loss
        patience_counter = 0  # Reset counter
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered!")
            break

Epoch 1/15 - Loss: 1.0377
Epoch 2/15 - Loss: 0.8076
Epoch 3/15 - Loss: 0.6648
Epoch 4/15 - Loss: 0.6362
Epoch 5/15 - Loss: 0.6210
Epoch 6/15 - Loss: 0.6246
Epoch 7/15 - Loss: 0.6171
Epoch 8/15 - Loss: 0.6154
Epoch 9/15 - Loss: 0.6180
Epoch 10/15 - Loss: 0.6138
Epoch 11/15 - Loss: 0.6148
Epoch 12/15 - Loss: 0.6225
Epoch 13/15 - Loss: 0.6121
Epoch 14/15 - Loss: 0.6147
Epoch 15/15 - Loss: 0.6167


In [24]:
best_loss

0.6121297030194291

In [25]:
from tqdm import tqdm

predictions = []

with torch.no_grad(): # Disables gradient tracking since we’re just predicting.
    for text in tqdm(texts, desc="Predicting genres"):
        encoded = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512).to("cuda")
        outputs = model(**encoded)
        probs = torch.sigmoid(outputs.logits)
        preds = (probs > 0.5).int().squeeze().tolist()
        predictions.append(preds)

df = pd.DataFrame(predictions, columns=genre_cols)
df = pd.concat([data[["Movie_ID", "Movie_Name"]], df], axis=1)

df.head()

Predicting genres: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1633/1633 [00:38<00:00, 42.12it/s]


,Movie_ID,Movie_Name,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,1,1,1,1,0,0,0,1,0,0,1,0,0,1,0,1,0
1,2,GoldenEye (1995),1,1,0,0,0,0,0,0,0,1,1,0,1,0,1,1,0,0
2,3,Four Rooms (1995),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,Get Shorty (1995),0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,1,0,0
4,5,Copycat (1995),1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0


In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score

genres_list = [
    "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", 
    "Documentary", "Drama", "Fantasy", "Film Noir", "Horror", "Musical", 
    "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]

# Initialize result dictionary
results = {"Genre": [], "Precision": [], "Recall": [], "F1-score": []}

# Compute metrics for each genre
for genre in genres_list:
    precision = precision_score(data[genre], df[genre])
    recall = recall_score(data[genre], df[genre])
    f1 = f1_score(data[genre], df[genre])

    # Store results rounded to 2 decimal places
    results["Genre"].append(genre)
    results["Precision"].append(round(precision, 2))
    results["Recall"].append(round(recall, 2))
    results["F1-score"].append(round(f1, 2))

# Convert results into a DataFrame
metrics_df = pd.DataFrame(results)

metrics_df.to_csv("DB_Classification_Score.csv", index=False)

# Display the result
print(metrics_df)

          Genre  Precision  Recall  F1-score
0        Action       0.40    0.84      0.54
1     Adventure       0.37    0.90      0.52
2     Animation       0.23    1.00      0.38
3    Children's       0.48    0.95      0.64
4        Comedy       0.64    0.76      0.69
5         Crime       0.25    0.85      0.39
6   Documentary       0.18    0.88      0.30
7         Drama       0.73    0.81      0.77
8       Fantasy       0.12    1.00      0.22
9     Film Noir       0.08    1.00      0.16
10       Horror       0.28    1.00      0.44
11      Musical       0.18    0.89      0.30
12      Mystery       0.16    0.90      0.27
13      Romance       0.32    0.71      0.45
14       Sci-Fi       0.31    0.96      0.47
15     Thriller       0.42    0.86      0.56
16          War       0.24    0.86      0.38
17      Western       0.18    0.96      0.30


In [28]:
model.save_pretrained("./distilbert_HPT_local")
tokenizer.save_pretrained("./distilbert_HPT_local")

('./distilbert_HPT_local\\tokenizer_config.json',
 './distilbert_HPT_local\\special_tokens_map.json',
 './distilbert_HPT_local\\vocab.txt',
 './distilbert_HPT_local\\added_tokens.json')